<a href="https://colab.research.google.com/github/narendra974/insidedeeplearning/blob/main/FC_Networks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Fully Connected Networks

To implement any kind of neural network in PyTorch we must phrase the problem as an optimization problem (function minimization)

1. Input Data and labels are feed into the process.
2. Output of the model is used with the true label to compute a loss.
3. quantifying how bad the model is doing with the loss function.
4. Compute the gradient of loss with each parameter. 
5. Update the parameters with the gradients calculated.

Function Minimization Problem

Alterthe parameters 'Theta' to minimize the error/lossof the neural network's prediction against the correct predictions over the entire dataset.

